# Step 1: Setup

In [2]:
! pip install datasets transformers accelerate evaluate peft


`datasets` — load classification datasets

`transformers` — model & tokenizer

`accelerate` — optimized training & memory management

`evaluate` — metrics calculation

`peft` — parameter-efficient fine-tuning library (for adapters, LoRA)

# Step 2: Dataset and Task
Example: Sentiment Analysis with IMDb dataset (binary classification).

In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")
train_ds = dataset["train"].shuffle(seed=42).select(range(2000))  # small subset for demo
test_ds = dataset["test"].shuffle(seed=42).select(range(500))


c:\Users\TCS\anaconda3\envs\newtorchenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 3: Tokenizer and Model Setup

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(preprocess, batched=True)
test_ds = test_ds.map(preprocess, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])


Map: 100%|██████████| 500/500 [00:00<00:00, 4393.72 examples/s]


# Step 4: Define Fine-tuning Variants
(a) Full Fine-tuning (baseline)

In [3]:
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results_full_ft",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    logging_steps=50,
    report_to=[],  # <-- disables wandb, tensorboard, etc.
)

import evaluate
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()
results_full_ft = trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\TCS\anaconda3\envs\newtorchenv\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/375 [00:00<?, ?it/s]c:\Users\TCS\anaconda3\envs\newtorchenv\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
 14%|█▎        | 51/375 [00:14<01:23,  3.90it/s]

{'loss': 0.5604, 'grad_norm': 9.413958549499512, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4}


 27%|██▋       | 100/375 [00:27<01:15,  3.65it/s]

{'loss': 0.4501, 'grad_norm': 2.2814810276031494, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}


                                                 
 33%|███▎      | 125/375 [00:35<01:02,  4.00it/s]

...

                                                 
100%|██████████| 375/375 [01:59<00:00,  3.93it/s]

{'eval_loss': 0.5904265642166138, 'eval_accuracy': 0.848, 'eval_runtime': 2.1405, 'eval_samples_per_second': 233.589, 'eval_steps_per_second': 7.475, 'epoch': 3.0}


100%|██████████| 375/375 [02:24<00:00,  2.60it/s]


{'train_runtime': 144.3296, 'train_samples_per_second': 41.572, 'train_steps_per_second': 2.598, 'train_loss': 0.26936251831054686, 'epoch': 3.0}


100%|██████████| 16/16 [00:01<00:00,  8.04it/s]


(b) Adapter Tuning with PEFT

In [4]:
from peft import get_peft_model, LoraConfig, TaskType

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,                # bottleneck dimension
    lora_alpha=16,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()
results_adapter = trainer.evaluate()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 14%|█▎        | 51/375 [00:08<00:51,  6.25it/s]

{'loss': 0.7016, 'grad_norm': 6.678351402282715, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4}


 27%|██▋       | 101/375 [00:16<00:43,  6.29it/s]

{'loss': 0.6954, 'grad_norm': 4.2916412353515625, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}


 33%|███▎      | 125/375 [00:22<00:39,  6.28it/s]

...

100%|██████████| 375/375 [01:09<00:00,  6.19it/s]

{'eval_loss': 0.660788357257843, 'eval_accuracy': 0.646, 'eval_runtime': 2.255, 'eval_samples_per_second': 221.73, 'eval_steps_per_second': 7.095, 'epoch': 3.0}


100%|██████████| 375/375 [01:09<00:00,  5.38it/s]


{'train_runtime': 69.7133, 'train_samples_per_second': 86.067, 'train_steps_per_second': 5.379, 'train_loss': 0.6823002217610677, 'epoch': 3.0}


100%|██████████| 16/16 [00:02<00:00,  7.71it/s]


# Step 5: Track Training Time & Memory Usage
Use Python's time module or accelerate's built-in profiling for timing. In this case manually looking at cell execution time

Monitored GPU memory with nvidia-smi

In [5]:
import time
start = time.time()
trainer.train()
end = time.time()
print(f"Training time: {end - start} seconds")


 14%|█▎        | 51/375 [00:08<00:51,  6.33it/s]

{'loss': 0.6536, 'grad_norm': 6.657367706298828, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4}


 27%|██▋       | 101/375 [00:16<00:43,  6.33it/s]

{'loss': 0.6079, 'grad_norm': 3.609138250350952, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}


 33%|███▎      | 125/375 [00:22<00:39,  6.31it/s]

...

{'loss': 0.409, 'grad_norm': 3.137791156768799, 'learning_rate': 3.3333333333333333e-06, 'epoch': 2.8}


100%|██████████| 375/375 [01:08<00:00,  6.28it/s]

{'eval_loss': 0.4089295566082001, 'eval_accuracy': 0.824, 'eval_runtime': 2.2602, 'eval_samples_per_second': 221.22, 'eval_steps_per_second': 7.079, 'epoch': 3.0}


100%|██████████| 375/375 [01:09<00:00,  5.42it/s]

{'train_runtime': 69.159, 'train_samples_per_second': 86.757, 'train_steps_per_second': 5.422, 'train_loss': 0.4866800816853841, 'epoch': 3.0}
Training time: 69.36183667182922 seconds


In [7]:
import torch

def print_gpu_memory_usage(label=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 3)
        reserved = torch.cuda.memory_reserved() / (1024 ** 3)
        print(f"{label} GPU memory allocated: {allocated:.2f} GB, reserved: {reserved:.2f} GB")
    else:
        print(f"{label} CUDA not available.")

# Example usage:
# Before training
print_gpu_memory_usage("Before training:")
# trainer.train()


model_full = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

trainer = Trainer(
    model=model_full,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()
# results_full_ft = trainer.evaluate()
print_gpu_memory_usage("After training (Adapter/PEFT):")
# to compare with full fine-tuning, you would repeat the above
# with a model that is NOT wrapped with PEFT/adapter, e.g.:
# trainer_full = Trainer(model=model_full, ...)
# print_gpu_memory_usage("Before training (Full Fine-tuning):")
# trainer_full.train()
# print_gpu_memory_usage("After training (Full Fine-tuning):")


Before training: GPU memory allocated: 0.43 GB, reserved: 1.55 GB


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 14%|█▎        | 51/375 [00:12<01:21,  3.96it/s]

{'loss': 0.5557, 'grad_norm': 2.8914098739624023, 'learning_rate': 4.3333333333333334e-05, 'epoch': 0.4}


 27%|██▋       | 101/375 [00:25<01:07,  4.05it/s]

{'loss': 0.4171, 'grad_norm': 3.123000383377075, 'learning_rate': 3.6666666666666666e-05, 'epoch': 0.8}


...

{'loss': 0.1103, 'grad_norm': 0.47726503014564514, 'learning_rate': 3.3333333333333333e-06, 'epoch': 2.8}


100%|██████████| 375/375 [02:26<00:00,  4.08it/s]

{'eval_loss': 0.6336063146591187, 'eval_accuracy': 0.838, 'eval_runtime': 3.0153, 'eval_samples_per_second': 165.823, 'eval_steps_per_second': 5.306, 'epoch': 3.0}


100%|██████████| 375/375 [02:39<00:00,  2.34it/s]

{'train_runtime': 159.9713, 'train_samples_per_second': 37.507, 'train_steps_per_second': 2.344, 'train_loss': 0.27567799313863117, 'epoch': 3.0}
After training (Adapter/PEFT): GPU memory allocated: 1.66 GB, reserved: 3.24 GB


# Step 6: Analyze Trade-offs
| Technique        | training_loss | Training Time (s) | Memory Usage (GB) | Notes                                                           |
|------------------|--------------|-------------------|-------------------|-----------------------------------------------------------------|
| Full Fine-tuning | 0.269        |144.33s               | 1.66                 | Trains all model parameters, best accuracy but expensive        |
| Adapter Tuning with PEFT  |  0.68         | 69.7               | 0.43                 | Trains small additional params, faster & less memory            |